Padding (패딩)
자연어 처리를 하다보면 각 문장의 길이는 다를 수 있다. 하지만 기계가 처리하는 방식은 행렬을 기준으로 처리하기 때문에 행렬연산을 위해서 패딩을 해줌으로서 길이를 맞추는 과정이 필요하다.

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

tokenizer = Tokenizer()

tokenizer.fit_on_texts(preprocessed_sentences)
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

maxlen = max(len(item) for item in encoded)
print('sentence 최대길이')

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]
sentence 최대길이


In [3]:
for sentence in encoded:
    while len(sentence) < maxlen:
        sentence.append(0)

pad_np = np.array(encoded)
print(pad_np)

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


위와 같이 하면 기계가 병렬처리를 수월하게 진행할 수 있다. 0으로 채우는 것을 zero-padding이라고 한다.
케라스에서는 위와 같은 패딩처리를 위해서 pad_sequences()를 제공하고 있다.


In [5]:
tokenizer.fit_on_texts(preprocessed_sentences)
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)

from tensorflow.keras.preprocessing.sequence import pad_sequences
padd = pad_sequences(encoded) #0이 앞으로 가게

print(padd)
padd = pad_sequences(encoded, padding='post') #0이 뒤로가게
print(padd)

[[ 0  0  0  0  0  1  5]
 [ 0  0  0  0  1  8  5]
 [ 0  0  0  0  1  3  5]
 [ 0  0  0  0  0  9  2]
 [ 0  0  0  2  4  3  2]
 [ 0  0  0  0  0  3  2]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  2]
 [ 7  7  3  2 10  1 11]
 [ 0  0  0  1 12  3 13]]
[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


위의 예제에서는 길이를 가장 긴 것에 맞춰놨지만, 모든 경우에서 가장 긴것에 맞추는건 아니다.
다음과 같이 맞춤 길이를 설정할 수 있다.

In [7]:
padd = pad_sequences(encoded,padding='post',maxlen=5) #이렇게 하면 앞에 두자리가 삭제되는 것을 볼 수 있다. 
print(padd)
padd = pad_sequences(encoded,padding='post',truncating='post',maxlen=5) #이렇게 하면 뒤에 두자리가 삭제되는 것을 볼 수 있다. 추가로 value = '값' 을 설정해서 0대신 사용할 값을 설정해줄 수 있다.
print(padd)

[[ 1  5  0  0  0]
 [ 1  8  5  0  0]
 [ 1  3  5  0  0]
 [ 9  2  0  0  0]
 [ 2  4  3  2  0]
 [ 3  2  0  0  0]
 [ 1  4  6  0  0]
 [ 1  4  6  0  0]
 [ 1  4  2  0  0]
 [ 3  2 10  1 11]
 [ 1 12  3 13  0]]
[[ 1  5  0  0  0]
 [ 1  8  5  0  0]
 [ 1  3  5  0  0]
 [ 9  2  0  0  0]
 [ 2  4  3  2  0]
 [ 3  2  0  0  0]
 [ 1  4  6  0  0]
 [ 1  4  6  0  0]
 [ 1  4  2  0  0]
 [ 7  7  3  2 10]
 [ 1 12  3 13  0]]


원-핫 인코딩(One-Hot encoding)
원핫인코딩을 배우기 앞서 단어 집합을 정의해보자 단어 집합은 서로 다른 단어들의 집합을 나타낸다. 
원핫인코딩을 하기위해서 단어집합을 만들어야한다. 다음에 단어 집합의 크기를 벡터의 차원으로 설정하고 그 인덱스에 1, 다른 인덱스에 0으로 표현하는 기법이다.


In [8]:
from konlpy.tag import Okt

okt = Okt()
token = okt.morphs('나는 자연어 처리를 배운다')
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [10]:
word_to_index = {word:index for index, word in enumerate(token)}
print(word_to_index)

def one_hot(word,word_to_index):
    one_hot_vec = [0]*len(word_to_index)
    index = word_to_index[word]
    one_hot_vec[index] = 1
    return one_hot_vec

ohe = one_hot('자연어',word_to_index)
print(ohe)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}
[0, 0, 1, 0, 0, 0]


다음은 keras를 이용한 원핫인코딩을 보여준다. to_categorical 을 사용한다.


In [17]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('keras 토크나이저 결과 : ',tokenizer.word_index)
#이렇게 토크나이저 한 것을 토대로 정제했다고 치고 sub_text를 만든 뒤 순서로만 나타내려면 texts_to_sequences를 사용한다.
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded = tokenizer.texts_to_sequences([sub_text])[0]
print(encoded)

ohe = to_categorical(encoded) #그리고 to_categorical 을 사용해서 원핫인코딩으로 만들 수 있다.
print(ohe)

keras 토크나이저 결과 :  {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}
[2, 5, 1, 6, 3, 7]
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


하지만 원핫인코딩을 사용하면 단어들의 유사성을 잃어버릴 수 있고 저장공간에 매우 비효율적이다.
따라서 단어의 잠재의미를 파악해서 벡터화하는 카운트기반 방법인 LSA(잠재의미분석), HAL 등이 있고 예측을 기반으로 벡터화하는 NNLM, RNNLM, Word2Vec, FastText등이 있다.